In [1]:
## No elimine esta celda, ya que permite actualizar los archivos .py
## sin tener que reiniciar el Kernel de Jupyter.
%load_ext autoreload
%autoreload 2
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torchvision.datasets import ImageFolder
import numpy as np
import torch
from train import find_best_model, run_experiment
from transforms import Transforms
from utils import  set_seed, count_instances_per_class
import os
import warnings


warnings.filterwarnings("ignore", message = "Palette images with Transparency expressed in bytes should be converted to RGBA images")
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache()

set_seed()


In [2]:
train_transforms = A.Compose(
    [A.Resize(256, 256), A.ToFloat(), ToTensorV2()]
)
validation_transforms = A.Compose(
    [A.Resize(256, 256), A.ToFloat(), ToTensorV2()]
)


train_data = ImageFolder(
    "house_plant_species/train",
    transform = Transforms(train_transforms),
)
validation_data = ImageFolder(
    "house_plant_species/validation",
    transform = Transforms(validation_transforms),
)



print(f"Elementos en Entrenamiento: {len(train_data)}")
print(f"Elementos en Validación: {len(validation_data)}")
print(f"There are {len(train_data.classes)} classes!")
#

# Llamar a las funciones
class_counts = count_instances_per_class(train_data)

# Calcular estadísticas
class_instance_counts = np.array(list(class_counts.values()))
mean_instances = np.mean(class_instance_counts)
std_instances = np.std(class_instance_counts)
min_instances = np.min(class_instance_counts)
max_instances = np.max(class_instance_counts)

# # Mostrar resultados
# print("\nInstancias por clase:\n")
# for class_name, count in class_counts.items():
#     print(f"Clase {class_name}: {count} imágenes\n===========================================")

print("\nEstadísticas de las instancias por clase:\n")
print(f"Promedio de instancias:\n============================== {mean_instances:.2f}")
print(f"Desviación estándar:\n============================== {std_instances:.2f}")
print(f"Cantidad mínima de instancias:\n============================== {min_instances}")
print(f"Cantidad máxima de instancias:\n============================== {max_instances}")

Elementos en Entrenamiento: 10835
Elementos en Validación: 2736
There are 47 classes!

Estadísticas de las instancias por clase:

Promedio de instancias:
============================== 230.53
Desviación estándar:
============================== 77.46
Cantidad mínima de instancias:
============================== 49
Cantidad máxima de instancias:
============================== 432


In [9]:
class_counts

defaultdict(int,
            {'African Violet (Saintpaulia ionantha)': 269,
             'Aloe Vera': 200,
             'Anthurium (Anthurium andraeanum)': 326,
             'Areca Palm (Dypsis lutescens)': 151,
             'Asparagus Fern (Asparagus setaceus)': 135,
             'Begonia (Begonia spp.)': 157,
             'Bird of Paradise (Strelitzia reginae)': 144,
             'Birds Nest Fern (Asplenium nidus)': 232,
             'Boston Fern (Nephrolepis exaltata)': 245,
             'Calathea': 188,
             'Cast Iron Plant (Aspidistra elatior)': 212,
             'Chinese Money Plant (Pilea peperomioides)': 305,
             'Chinese evergreen (Aglaonema)': 411,
             'Christmas Cactus (Schlumbergera bridgesii)': 249,
             'Chrysanthemum': 132,
             'Ctenanthe': 176,
             'Daffodils (Narcissus spp.)': 292,
             'Dracaena': 161,
             'Dumb Cane (Dieffenbachia spp.)': 432,
             'Elephant Ear (Alocasia spp.)': 265,
     

### Best architecture is **efficientnet_b5.sw_in12k** obtained with:

###### "color_jitter", "horizontal_flip", "brightness_contrast", "blur"
###### Trivial fc_layers => Lazylinear(512) => Linear(512, num_clases)
###### Resize to 256 x 256 images
###### ToFloat() and not Normalize

### **New Change**
####         for layer in [self.backbone.conv_head, self.backbone.bn2, self.backbone.global_pool]:
####            for param in layer.parameters():
####                param.requires_grad = True
####  416 x 416 input images resizing


In [ ]:
run_experiment(
    selected_transforms = ["center_crop", "color_jitter", "horizontal_flip", "brightness_contrast", "noise", "blur"],
    model_name = "efficientnet_b5.sw_in12k",
    base_name = "",
    extra_info = "bn_drp_unfreeze_416", 
    model_type = "efficientnet_b5.sw_in12k",
    num_classes = 47,
    training_params = {"learning_rate": 0.0003, "batch_size": 32, "num_epochs": 50},
    frozen = True,
    result_file = "results/efficient_net.json",
    normalize = False,
    use_float = True,
    resize_size = 416  # Aquí defines dinámicamente el tamaño de `resize`
)

Running experiment: efficientnet_b5.sw_in12k__bn_drp_unfreeze_416
Using device: cuda


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: joseflores1 (benjaminrromo-universidad-adolfo-ib-ez). Use `wandb login --relogin` to force relogin


  2%|▏         | 1/50 [02:41<2:11:38, 161.20s/it]

Epoch: 1/50 - Time: 161.02s - Train Loss: 1.3811, Train Accuracy: 0.7659 - Validation Loss: 0.4153, Validation Accuracy: 0.9291
Model checkpoint saved at epoch 1


  4%|▍         | 2/50 [05:22<2:08:53, 161.11s/it]

...

Epoch: 11/50 - Time: 165.08s - Train Loss: 0.0740, Train Accuracy: 0.9794 - Validation Loss: 0.2012, Validation Accuracy: 0.9448


 22%|██▏       | 11/50 [32:40<1:55:49, 178.20s/it]

Epoch: 12/50 - Time: 164.22s - Train Loss: 0.0573, Train Accuracy: 0.9843 - Validation Loss: 0.2093, Validation Accuracy: 0.9470
Early stopping triggered at epoch 6. Best Validation Loss: 0.1831


elapsed_time,▁▁▁▄▃▇▅█▇██▇
epoch,▁▂▂▃▄▄▅▅▆▇▇█
train_acc,▁▆▇▇████████
train_loss,█▃▂▂▁▁▁▁▁▁▁▁
val_acc,▁▅▅▇▆█▆▇█▅▅▆
val_loss,█▃▂▁▁▁▁▁▁▂▂▂
elapsed_time,164.22048
epoch,12
train_acc,0.98428
train_loss,0.05734
val_acc,0.947


Results to save: {'architecture': 'efficientnet_b5.sw_in12k__bn_drp_unfreeze_416', 'model_type': 'efficientnet_b5.sw_in12k', 'best_epoch': 6, 'best_val_loss': 0.1831230824369244, 'best_train_loss': 0.10781560898565594, 'max_val_acc': 0.9565058350563049, 'test_accuracy': 0.0, 'training_params': {'learning_rate': 0.0003, 'batch_size': 32, 'num_epochs': 50}}
Resultados guardados correctamente en results/efficient_net.json
Experimento terminado para efficientnet_b5.sw_in12k__bn_drp_unfreeze_416


### *New Color_Jitter*:

#### Changed color_jitter to : "color_jitter": A.ColorJitter(brightness = 0.4, contrast = 0.4, saturation = 0.3, hue = 0.2, p = 0.7)
#### from "color_jitter": A.ColorJitter(brightness = 0.2, contrast = 0.2, saturation = 0.2, hue = 0.1, p = 0.5),
#### No center crop, added random_rotate in 15 degrees

In [2]:
run_experiment(
    selected_transforms = ["color_jitter", "horizontal_flip", "brightness_contrast", "noise", "blur", "random_rotate"],
    model_name = "efficientnet_b5.sw_in12k",
    base_name = "",
    extra_info = "bn_drp_unfreeze_416_cj_cp_b64_ff", 
    model_type = "efficientnet_b5.sw_in12k",
    num_classes = 47,
    training_params = {"learning_rate": 0.0003, "batch_size": 64, "num_epochs": 50},
    frozen = True,
    result_file = "results/efficient_net.json",
    normalize = False,
    use_float = True,
    resize_size = 416  # Aquí defines dinámicamente el tamaño de `resize`
)

Running experiment: efficientnet_b5.sw_in12k__bn_drp_unfreeze_416_cj_cp_b64_ff


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Using device: cuda


wandb: Currently logged in as: joseflores1 (benjaminrromo-universidad-adolfo-ib-ez). Use `wandb login --relogin` to force relogin


  2%|▏         | 1/50 [02:44<2:14:35, 164.81s/it]

...

Epoch: 14/50 - Time: 167.55s - Train Loss: 0.8418, Train Accuracy: 0.7848 - Validation Loss: 0.1659, Validation Accuracy: 0.9572


 28%|██▊       | 14/50 [41:37<1:47:03, 178.43s/it]

Epoch: 15/50 - Time: 167.55s - Train Loss: 0.8141, Train Accuracy: 0.7924 - Validation Loss: 0.1662, Validation Accuracy: 0.9561
Early stopping triggered at epoch 8. Best Validation Loss: 0.1555


elapsed_time,▃▁▂▂▅▇█▆▆▆▇▆▅▇▇
epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
train_acc,▁▆▇▇█▇█████████
train_loss,█▃▂▂▁▂▁▁▁▁▁▁▁▁▁
val_acc,▁▅▇▇▇▇▇███▇▇▇██
val_loss,█▄▂▂▁▁▁▁▁▁▁▁▁▁▁
elapsed_time,167.55137
epoch,15
train_acc,0.79244
train_loss,0.81414
val_acc,0.95614


Results to save: {'architecture': 'efficientnet_b5.sw_in12k__bn_drp_unfreeze_416_cj_cp_b64_ff', 'model_type': 'efficientnet_b5.sw_in12k', 'best_epoch': 8, 'best_val_loss': 0.1554810376621263, 'best_train_loss': 0.86472033255199, 'max_val_acc': 0.9590643048286438, 'test_accuracy': 0.0, 'training_params': {'learning_rate': 0.0003, 'batch_size': 64, 'num_epochs': 50}}
Resultados guardados correctamente en results/efficient_net.json
Experimento terminado para efficientnet_b5.sw_in12k__bn_drp_unfreeze_416_cj_cp_b64_ff


In [3]:
run_experiment(
    selected_transforms = ["color_jitter", "horizontal_flip", "brightness_contrast", "noise", "blur", "random_rotate"],
    model_name = "efficientnet_b5.sw_in12k",
    base_name = "",
    extra_info = "bn_drp_unfreeze_416_cj_cp_b84_ff", 
    model_type = "efficientnet_b5.sw_in12k",
    num_classes = 47,
    training_params = {"learning_rate": 0.0003, "batch_size": 84, "num_epochs": 50},
    frozen = True,
    result_file = "results/efficient_net.json",
    normalize = False,
    use_float = True,
    resize_size = 416  # Aquí defines dinámicamente el tamaño de `resize`
)

Running experiment: efficientnet_b5.sw_in12k__bn_drp_unfreeze_416_cj_cp_b84_ff


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Using device: cuda


wandb: Currently logged in as: joseflores1 (benjaminrromo-universidad-adolfo-ib-ez). Use `wandb login --relogin` to force relogin


Initial max val_accuracy: 0.9591, Second best: 0.9587



...

Epoch: 36/50 - Time: 183.37s - Train Loss: 0.7231, Train Accuracy: 0.8168 - Validation Loss: 0.1859, Validation Accuracy: 0.9583


 72%|███████▏  | 36/50 [1:51:32<43:22, 185.89s/it]

Epoch: 37/50 - Time: 180.55s - Train Loss: 0.7972, Train Accuracy: 0.7993 - Validation Loss: 0.1938, Validation Accuracy: 0.9554
Early stopping triggered at epoch 22. Best Validation Loss: 0.1544


elapsed_time,▅▁▃▄▅▆▅▅▅▅▅▄▄▅▄▅▄▅▅▆▅▅▅▆▅▆▇▄▆▆▅▅▆▅▃█▅
epoch,▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇███
train_acc,▁▆▆▇▇▇▇▇█████▇▇▇▇▇██▇▇██▇███▇▇▇▇████▇
train_loss,█▄▃▂▂▂▂▂▁▁▁▁▁▁▁▂▁▁▁▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_acc,▁▅▆▇▇▇▇▇███▇█▇██▇▇██▇██▇▇▇█▇▇▇▇▇▇▇▇██
val_loss,█▄▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂▂▁▂▁▂
elapsed_time,180.54506
epoch,37
train_acc,0.79929
train_loss,0.79723
val_acc,0.95541


Results to save: {'architecture': 'efficientnet_b5.sw_in12k__bn_drp_unfreeze_416_cj_cp_b84_ff', 'model_type': 'efficientnet_b5.sw_in12k', 'best_epoch': 11, 'best_val_loss': 0.15440661564582225, 'best_train_loss': 0.7508407854475081, 'max_val_acc': 0.9597952961921692, 'test_accuracy': 0.0, 'training_params': {'learning_rate': 0.0003, 'batch_size': 84, 'num_epochs': 50}}
Resultados guardados correctamente en results/efficient_net.json
Experimento terminado para efficientnet_b5.sw_in12k__bn_drp_unfreeze_416_cj_cp_b84_ff


In [2]:
run_experiment(
    selected_transforms = ["color_jitter", "horizontal_flip", "brightness_contrast", "noise", "blur", "random_rotate"],
    model_name = "efficientnet_b5.sw_in12k",
    base_name = "",
    extra_info = "bn_drp_unfreeze_416_cj_cp_b88_ff", 
    model_type = "efficientnet_b5.sw_in12k",
    num_classes = 47,
    training_params = {"learning_rate": 0.0003, "batch_size": 88, "num_epochs": 50},
    frozen = True,
    result_file = "results/efficient_net.json",
    normalize = False,
    use_float = True,
    resize_size = 416  # Aquí defines dinámicamente el tamaño de `resize`
)

Running experiment: efficientnet_b5.sw_in12k__bn_drp_unfreeze_416_cj_cp_b88_ff


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Using device: cuda


wandb: Currently logged in as: joseflores1 (benjaminrromo-universidad-adolfo-ib-ez). Use `wandb login --relogin` to force relogin


Initial max val_accuracy: 0.9598, Second best: 0.9591



...

Epoch: 26/50 - Time: 176.70s - Train Loss: 0.7967, Train Accuracy: 0.7975 - Validation Loss: 0.1948, Validation Accuracy: 0.9529


 52%|█████▏    | 26/50 [1:21:21<1:15:06, 187.77s/it]

Epoch: 27/50 - Time: 181.14s - Train Loss: 0.7757, Train Accuracy: 0.8016 - Validation Loss: 0.1875, Validation Accuracy: 0.9496
Early stopping triggered at epoch 12. Best Validation Loss: 0.1471


elapsed_time,▆▄▅▇▇█▇▆█▇▆▆▅▆▆▅▅▆▆▅▇▅▆▇▅▁▆
epoch,▁▁▂▂▂▂▃▃▃▃▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇██
train_acc,▁▆▇▆▇▇█▇███████████████████
train_loss,█▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_acc,▁▆▇▇▇▇▇▇██████▇█▇██▇▇▇██▇▇▇
val_loss,█▄▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▂▂
elapsed_time,181.13906
epoch,27
train_acc,0.80164
train_loss,0.77572
val_acc,0.94956


Results to save: {'architecture': 'efficientnet_b5.sw_in12k__bn_drp_unfreeze_416_cj_cp_b88_ff', 'model_type': 'efficientnet_b5.sw_in12k', 'best_epoch': 11, 'best_val_loss': 0.14713793000555597, 'best_train_loss': 0.7795222195667949, 'max_val_acc': 0.9608917832374573, 'test_accuracy': 0.0, 'training_params': {'learning_rate': 0.0003, 'batch_size': 88, 'num_epochs': 50}}
Resultados guardados correctamente en results/efficient_net.json
Experimento terminado para efficientnet_b5.sw_in12k__bn_drp_unfreeze_416_cj_cp_b88_ff


In [3]:
file_path = "results/efficient_net.json"
criteria = ["max_val_acc", "best_val_loss"]
 
for crit in criteria:
    print(f"El mejor modelo según {crit} es {find_best_model(file_path, crit)["architecture"]} con {crit} de {find_best_model(file_path, crit)[f"{crit}"]}")

El mejor modelo según max_val_acc es efficientnet_b5.sw_in12k__bn_drp_unfreeze_416_cj_cp_b88_ff con max_val_acc de 0.9608917832374573
El mejor modelo según best_val_loss es efficientnet_b5.sw_in12k__bn_drp_unfreeze_416_cj_cp_b88_ff con best_val_loss de 0.14713793000555597
